# Load testing dataset

In [1]:
import json
with open("./dict/clean_th_lyrics.json", encoding="utf8") as f:
    test_input = json.loads(f.read())

print(len(test_input))

930


In [2]:
with open("./dict/clean_en_lyrics.json", encoding="utf8") as f:
    test_output = json.loads(f.read())

print(len(test_output))

930


# Import module

In [3]:
from model import sefr_cut
from model.TH2IPA import TH2IPA
from model.IPA2ENG import IPA_matching
from neuspell import BertChecker
import re

data folder is set to `e:\nlp\nlp-env\neuspell\neuspell\../data` script


# Initial variables

In [4]:
th2ipa = TH2IPA()
checker = BertChecker()

sefr_cut.SEFR_CUT.load_model(engine='model')
checker.from_pretrained()

loading model.....
Success
loading vocab from path:e:\nlp\nlp-env\neuspell\neuspell\../data/checkpoints/subwordbert-probwordnoise\vocab.pkl
initializing model
Number of parameters in the model: 185211810
Loading model params from checkpoint dir: e:\nlp\nlp-env\neuspell\neuspell\../data/checkpoints/subwordbert-probwordnoise


In [ ]:
tokenize_result = []
th2ipa_result = []
ipa2eng_result = []
corrector_result = []

all_word = 0
correct_word = 0
incorrect_word = 0

# Pipeline

In [7]:
def integration_pipeline(th_sent, th2ipa, checker):
    token_list = sefr_cut.tokenize(th_sent, k=40)[0]
    tokenize_result.append(token_list)
    ipa_list = th2ipa(token_list)
    th2ipa_result.append(ipa_list)
    eng_sent = ' '.join([IPA_matching(i)[0] for i in ipa_list]) 
    ipa2eng_result.append(eng_sent)
    correct_sent = checker.correct(eng_sent)
    correct_sent = re.sub(' \' ', '\'', correct_sent)
    corrector_result.append(correct_sent)
    return correct_sent

In [8]:
def evaluate(all_true, all_pred):
    for i in range(len(all_true)):
        sent_true = all_true[i]
        sent_pred = all_pred[i]

        sent_pred = re.sub(' \' ', '\'', sent_pred)
        sent_pred = re.sub(' \'', '\'', sent_pred)
        
        word_true = sent_true.split()
        word_pred = sent_pred.split()

        for j in range(word_true):
            if word_true[j] == word_pred[j]:
                correct_word += 1
            else:
                incorrect_word += 1
            all_word += 1
        
        accuracy = correct_word / all_word
        print('accuracy:', accuracy)
        return accuracy

# Test

In [9]:
for i in test_input[:2]:
    integration_pipeline(test_input[0], th2ipa, checker)

corrector_result

1/1 [==============================] - 0s 177ms/step


In [ ]:
evaluate(test_output, corrector_result)